# Extraction of topics from Wikipedia pages

In [25]:
import IPython
import numpy as np
import networkx as nx
import requests
import pandas as pd
import csv
import matplotlib.pyplot as plt
import time
from fonctions import *
pd.set_option('mode.chained_assignment', None)

path = 'Results/EN_20180816_20180831/'

In [2]:
### Importing the nodes as a DataFrame for easier manipulations ###

dataFrame = dataFrame = pd.read_csv(path + 'nodes.csv', encoding='utf-8')
dataFrame['Label'] = dataFrame['Label'].astype('string')
print(dataFrame.info())
display(dataFrame)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11099 entries, 0 to 11098
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Id                          11099 non-null  int64  
 1   Label                       11099 non-null  string 
 2   modularity_class            11099 non-null  int64  
 3   Degree                      11099 non-null  int64  
 4   Eccentricity                11099 non-null  int64  
 5   closnesscentrality          11099 non-null  float64
 6   harmonicclosnesscentrality  11099 non-null  float64
 7   betweenesscentrality        11099 non-null  float64
dtypes: float64(3), int64(4), string(1)
memory usage: 693.8 KB
None


,Id,Label,modularity_class,Degree,Eccentricity,closnesscentrality,harmonicclosnesscentrality,betweenesscentrality
0,1492374,Nash-Healey,9,1,30,0.067653,0.072100,0.000000
1,81100,Rio_Bravo_(film),55,6,25,0.082406,0.092150,98264.927091
2,58216352,Shreya_Dhanwanthary,56,2,28,0.081431,0.087879,11097.000000
3,47375338,Anshu_Gupta,68,1,25,0.091960,0.101828,0.000000
4,41725258,Shooting_at_the_Asian_Games,19,8,26,0.086504,0.098272,179587.233729
...,...,...,...,...,...,...,...,...
11094,3256715,GoldenEye_(song),6,1,28,0.070957,0.080006,0.000000
11095,189981,Symphony_No._8_(Mahler),24,1,29,0.068878,0.073930,0.000000
11096,237925,Leopold_von_Ranke,11,4,26,0.088908,0.097874,38977.368145
11097,520449,Pacific_Plate,42,1,33,0.053110,0.055809,0.000000


In [3]:
### Extract nodes labels from Wikipedia queries ###



# URL for the quieries
urls = "https://en.wikipedia.org/w/api.php?action=query&prop=pageprops&format=json&pageids="


i=0
Nb_pages = len(dataFrame)


### Associate the 'Qid' value of Wikipedia pages in the DataFrame ###
### Give '-1' value if an error has occured during the query      ###

for pageId in np.int64(dataFrame['Id']):
    response = requests.get(urls + str(pageId)).json()
    try:
        Qid = (list(findkeys(response, 'wikibase_item'))[0])
        Title = (list(findkeys(response, 'title'))[0])
    except IndexError:
        dataFrame.loc[dataFrame['Id'] == pageId, 'Qid'] = '-1'
        i+=1
        print(i,"/", Nb_pages, dataFrame.loc[dataFrame['Id'] == pageId, 'Label'], "\t error")
    else:
        dataFrame.loc[dataFrame['Id'] == pageId, 'Qid'] = Qid
        dataFrame.loc[dataFrame['Id'] == pageId, 'Label'] = Title
        i+=1
        print(i,"/", Nb_pages, dataFrame.loc[dataFrame['Id'] == pageId, 'Label'].values[0])

# Clear the loading display
IPython.display.clear_output(wait=False)

        
# Save the DataFrame with their associated Qids
dataFrame.to_csv(path + 'pages_Qids.csv', encoding='utf-8')

dataFrame

,Id,Label,modularity_class,Degree,Eccentricity,closnesscentrality,harmonicclosnesscentrality,betweenesscentrality,Qid
0,1492374,Nash-Healey,9,1,30,0.067653,0.072100,0.000000,Q1965679
1,81100,Rio Bravo (film),55,6,25,0.082406,0.092150,98264.927091,Q1008351
2,58216352,Shreya Dhanwanthary,56,2,28,0.081431,0.087879,11097.000000,Q56240001
3,47375338,Anshu Gupta,68,1,25,0.091960,0.101828,0.000000,Q20742881
4,41725258,Shooting at the Asian Games,19,8,26,0.086504,0.098272,179587.233729,Q17125181
...,...,...,...,...,...,...,...,...,...
11094,3256715,GoldenEye (song),6,1,28,0.070957,0.080006,0.000000,Q2344401
11095,189981,Symphony No. 8 (Mahler),24,1,29,0.068878,0.073930,0.000000,Q270853
11096,237925,Leopold von Ranke,11,4,26,0.088908,0.097874,38977.368145,Q77184
11097,520449,Pacific Plate,42,1,33,0.053110,0.055809,0.000000,Q205411


In [4]:
### Extracting topic pages from database API ###


dataFrame = pd.read_csv(path + 'pages_Qids.csv')

# Match_topic_API = pd.DataFrame(columns = ['Qid', 'topic', 'probability', 'page_id', 'page_title', 'modularity_class', 'wiki_db'])
Match_topic_API = pd.DataFrame()


                
### The API's URL from the topic is extracted ###         
urls = "http://86.119.25.229:5000/api/v1/wikidata/topic?qid="
threshold = '0.1'

    
tmps1=time.time()
    
a=0 
n = dataFrame.index[-1]
for pageIndex in dataFrame.index:
    pageQid = dataFrame.at[pageIndex, 'Qid']
    pageModularity = dataFrame.at[pageIndex, 'modularity_class']
    response = requests.get(urls + pageQid + "&threshold=" + threshold).json()
    scores = list(findkeys(response, 'score'))
    topics = list(findkeys(response, 'topic'))
    
    try:
        page_title = response['name']
    except KeyError:
        page_title = dataFrame.at[pageIndex, 'Label']
    for i in range(len(scores)):
        page = dataFrame.iloc[pageIndex]
        page['Topic'] = topics[i]
        page['Probability'] = scores[i]
        Match_topic_API = Match_topic_API.append(page, ignore_index=True)
    print(a, "/",n)
    a+=1
    
    
# Clear the loading display
IPython.display.clear_output(wait=False)
tmps2=time.time()
print("Execution time = ",(tmps2-tmps1), " s")

# Save the results 
Match_topic_API.drop(columns = ['Unnamed: 0'], inplace = True )
Match_topic_API.to_csv(path + 'pages_topic.csv', encoding='utf-8')

Match_topic_API

Execution time =  4601.256925582886  s


,Degree,Eccentricity,Id,Label,Probability,Qid,Topic,betweenesscentrality,closnesscentrality,harmonicclosnesscentrality,modularity_class
0,1.0,30.0,1492374.0,Nash-Healey,0.967909,Q1965679,History_and_Society.Transportation,0.000000,0.067653,0.072100,9.0
1,1.0,30.0,1492374.0,Nash-Healey,0.956644,Q1965679,STEM.Engineering,0.000000,0.067653,0.072100,9.0
2,1.0,30.0,1492374.0,Nash-Healey,0.951152,Q1965679,STEM.STEM*,0.000000,0.067653,0.072100,9.0
3,1.0,30.0,1492374.0,Nash-Healey,0.187143,Q1965679,History_and_Society.Business_and_economics,0.000000,0.067653,0.072100,9.0
4,1.0,30.0,1492374.0,Nash-Healey,0.000000,Q1965679,Geography.Regions.Europe.Europe*,0.000000,0.067653,0.072100,9.0
...,...,...,...,...,...,...,...,...,...,...,...
44808,34.0,22.0,52261973.0,Thirteen Soldiers,0.692652,Q28129011,Culture.Media.Media*,13827.483322,0.120520,0.152254,54.0
44809,34.0,22.0,52261973.0,Thirteen Soldiers,0.592677,Q28129011,Culture.Media.Books,13827.483322,0.120520,0.152254,54.0
44810,34.0,22.0,52261973.0,Thirteen Soldiers,0.112805,Q28129011,History_and_Society.Politics_and_government,13827.483322,0.120520,0.152254,54.0
44811,34.0,22.0,52261973.0,Thirteen Soldiers,0.109716,Q28129011,History_and_Society.Military_and_warfare,13827.483322,0.120520,0.152254,54.0


In [22]:
df = pd.read_csv(path + 'pages_topic.csv', index_col = 'Unnamed: 0')

# src = pd.read_csv('Results/betweenness_centrality.csv')
# df = add_column(src, df, 'betweenesscentrality')

modularity_df = pd.DataFrame({'Main_Topics':[], 'Higher_Weighted_Article':[]})
# modularity_df = pd.DataFrame()

df = count_topic(df)
df = weight_topic(df)
# df = weight_topic_series(df)


df.sort_values(by = ['modularity_class', 'Weight'], inplace=True, ascending = [True, False])
# display(df)


total_weight = df[['Weight', 'modularity_class']].groupby(['modularity_class']).sum()


nb_topic = int( df['modularity_class'].max() ) + 1
for i in range(nb_topic):
    
    
    df.loc[df['modularity_class'] == i, 'Total Weight'] = total_weight.iloc[i].values
    df['Ratio Weight'] = df['Weight'] / df['Total Weight']
    
    
    list_topic = []
    str_main_topics = ''
    idx_topic = 0
    max_idx_topic = len(df.loc[df['modularity_class'] == i])
    while ( (len(list_topic) < 3) and (idx_topic < max_idx_topic) ):
        str_topic = df.loc[df['modularity_class'] == i, 'Subtopic'].values[idx_topic]
#         if ( (str_topic not in list_topic) and ( df.loc[df['Subtopic'] == str_topic, 'Degree Ratio'].values[0] >= 0.1) ):
        if (str_topic not in list_topic):

            list_topic.append(str_topic)
            str_main_topics +=  '/ ' + str_topic + ' (' + str((df.loc[(df['modularity_class'] == i) & (df['Subtopic'] == str_topic), 'Degree Ratio'].values[0]).round(decimals=2)) + ') '
        idx_topic += 1
    str_main_topics = str_main_topics[2:]
    
    
    
#     str_main_topics = list_topic[0] + ' (' + str((df.loc[(df['modularity_class'] == i) & (df['Subtopic'] == list_topic[0]), 'Degree Ratio'].values[0]).round(decimals=2)) + ') '
#     str_main_topics += '/ ' + list_topic[1] + ' (' + str((df.loc[(df['modularity_class'] == i) & (df['Subtopic'] == list_topic[1]), 'Degree Ratio'].values[0]).round(decimals=2)) + ') '
#     str_main_topics += '/ ' + list_topic[2] + ' (' + str((df.loc[(df['modularity_class'] == i) & (df['Subtopic'] == list_topic[2]), 'Degree Ratio'].values[0]).round(decimals=2)) + ') '
        
    page_id_max_weight = df.loc[df['modularity_class'] == i, 'Weight'].idxmax()
    str_weight = df.loc[page_id_max_weight, 'Label'] + ' (' + str(int(df.loc[page_id_max_weight, 'Weight'])) + ')'
        
    mod = {'Main_Topics':str_main_topics, 'Higher_Weighted_Article':str_weight}
    modularity_df = modularity_df.append(mod, ignore_index=True)


    
display(df)    

display(modularity_df)

df.to_csv(path + 'pages_main_topic.csv', encoding='utf-8')

modularity_df.to_csv(path + 'topic_by_degree.csv', encoding='utf-8')



,Degree,Eccentricity,Id,Label,Probability,Qid,Topic,betweenesscentrality,closnesscentrality,harmonicclosnesscentrality,modularity_class,Count,Total,Degree Ratio,Subtopic,Weight,Total Weight,Ratio Weight
197,3.0,24.0,2041030.0,Pacific saury,0.996303,Q1195079,STEM.STEM*,44379.0,0.098609,0.112094,0.0,7.0,34.0,0.205882,STEM*,20.922358,106.547539,0.196366
5950,2.0,25.0,28707242.0,Scallop aquaculture,0.949679,Q3373930,STEM.STEM*,22192.0,0.089761,0.099468,0.0,7.0,34.0,0.205882,STEM*,13.295511,106.547539,0.124785
196,3.0,24.0,2041030.0,Pacific saury,0.998362,Q1195079,STEM.Biology,44379.0,0.098609,0.112094,0.0,3.0,34.0,0.088235,Biology,8.985254,106.547539,0.084331
8681,3.0,23.0,491120.0,Fishing industry,0.901931,Q635139,History_and_Society.Business_and_economics,77647.0,0.109386,0.129155,0.0,3.0,34.0,0.088235,Business_and_economics,8.117376,106.547539,0.076185
21001,1.0,27.0,20917738.0,Pecten maximus,0.999440,Q1249634,STEM.STEM*,0.0,0.076102,0.081681,0.0,7.0,34.0,0.205882,STEM*,6.996078,106.547539,0.065662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44458,3.0,29.0,1578115.0,Selma Björnsdóttir,0.122533,Q175119,Culture.Performing_arts,0.5,0.068307,0.073372,77.0,2.0,117.0,0.017094,Performing_arts,0.735199,3150.654032,0.000233
35758,1.0,29.0,562259.0,List of eponymously named diseases,0.228166,Q11758019,History_and_Society.Society,0.0,0.067830,0.072338,77.0,3.0,117.0,0.025641,Society,0.684499,3150.654032,0.000217
14269,1.0,30.0,1882510.0,Wheel gymnastics,0.268951,Q567014,Culture.Performing_arts,0.0,0.064936,0.068902,77.0,2.0,117.0,0.017094,Performing_arts,0.537903,3150.654032,0.000171
40674,1.0,28.0,896415.0,List of political term limits,0.100889,Q6634307,Culture.Sports,0.0,0.073277,0.078442,77.0,5.0,117.0,0.042735,Sports,0.504443,3150.654032,0.000160


,Main_Topics,Higher_Weighted_Article
0,STEM* (0.21) / Biology (0.09) / Business_and_e...,Pacific saury (20)
1,History (0.18) / Military_and_warfare (0.18) /...,Robert the Bruce (660)
2,Politics_and_government (0.27) / Business_and_...,Cynthia Lummis (268)
3,Sports (0.9) / Society (0.02) / STEM* (0.01),Beauden Barrett (1584)
4,STEM* (0.17) / Society (0.15) / Biology (0.08),Neanderthal (752)
...,...,...
73,Military_and_warfare (0.26) / Society (0.19) /...,Northern Fleet (91)
74,Military_and_warfare (0.12) / Politics_and_gov...,New York and New Jersey campaign (1659)
75,Education (0.38) / STEM* (0.12) / Politics_and...,London Metropolitan University (422)
76,Media* (0.45) / Music (0.43) / Entertainment (...,Never Fade (Alice in Chains song) (347)


In [26]:
df = pd.read_csv(path + 'pages_topic.csv', index_col = 'Unnamed: 0')


modularity_df = pd.DataFrame({'Main_Topics':[], 'Higher_Weighted_Article':[]})

df = count_topic(df)
df = weight_topic(df)


df.sort_values(by = ['modularity_class', 'Weight'], inplace=True, ascending = [True, False])


df.to_csv(path + 'pages_main_topic.csv', encoding='utf-8')

total_weight = df[['Weight', 'modularity_class']].groupby(['modularity_class']).sum()
total_weight = total_weight.reindex(pd.RangeIndex(total_weight.index.max() + 1)).fillna(value=0)

nb_topic = int( df['modularity_class'].max() ) + 1
for i in range(nb_topic):
    
    # If the cluster has no topic or only Biography/Geography. Ignored for other cases
    if (len(df.loc[df['modularity_class'] == i]) == 0):
        mod = {'Main_Topics':'Biography/Geography', 'Higher_Weighted_Article':''}
        modularity_df = modularity_df.append(mod, ignore_index=True)
        continue 
        
    
    df.loc[df['modularity_class'] == i, 'Total Weight'] = total_weight.iloc[i].values
    df['Ratio Weight'] = df['Weight'] / df['Total Weight']
    
    
    list_topic = []
    str_main_topics = ''
    idx_topic = 0
    max_idx_topic = len(df.loc[df['modularity_class'] == i])
    while ( (len(list_topic) < 3) and (idx_topic < max_idx_topic) ):
        str_topic = df.loc[df['modularity_class'] == i, 'Subtopic'].values[idx_topic]
#         if ( (str_topic not in list_topic) and ( df.loc[df['Subtopic'] == str_topic, 'Degree Ratio'].values[0] >= 0.1) ):
        if (str_topic not in list_topic):
            list_topic.append(str_topic)
            str_main_topics +=  '/ ' + str_topic + ' (' + str((df.loc[(df['modularity_class'] == i) & (df['Subtopic'] == str_topic), 'Degree Ratio'].values[0]).round(decimals=2)) + ') '
        idx_topic += 1
    str_main_topics = str_main_topics[2:]
    
    
    
  
    page_id_max_weight = df.loc[df['modularity_class'] == i, 'Weight'].idxmax()
    str_weight = df.loc[page_id_max_weight, 'Label'] + ' (' + str(int(df.loc[page_id_max_weight, 'Weight'])) + ')'
        
    mod = {'Main_Topics':str_main_topics, 'Higher_Weighted_Article':str_weight}
    modularity_df = modularity_df.append(mod, ignore_index=True)


    
display(df)    

display(modularity_df)

df.to_csv(path + 'pages_main_topic.csv', encoding='utf-8')

modularity_df.to_csv(path + 'topic_by_degree.csv', encoding='utf-8')

,Degree,Eccentricity,Id,Label,Probability,Qid,Topic,betweenesscentrality,closnesscentrality,harmonicclosnesscentrality,modularity_class,Count,Total,Degree Ratio,Subtopic,Weight,Total Weight,Ratio Weight
197,3.0,24.0,2041030.0,Pacific saury,0.996303,Q1195079,STEM.STEM*,44379.0,0.098609,0.112094,0.0,7.0,34.0,0.205882,STEM*,20.922358,106.547539,0.196366
5950,2.0,25.0,28707242.0,Scallop aquaculture,0.949679,Q3373930,STEM.STEM*,22192.0,0.089761,0.099468,0.0,7.0,34.0,0.205882,STEM*,13.295511,106.547539,0.124785
196,3.0,24.0,2041030.0,Pacific saury,0.998362,Q1195079,STEM.Biology,44379.0,0.098609,0.112094,0.0,3.0,34.0,0.088235,Biology,8.985254,106.547539,0.084331
8681,3.0,23.0,491120.0,Fishing industry,0.901931,Q635139,History_and_Society.Business_and_economics,77647.0,0.109386,0.129155,0.0,3.0,34.0,0.088235,Business_and_economics,8.117376,106.547539,0.076185
21001,1.0,27.0,20917738.0,Pecten maximus,0.999440,Q1249634,STEM.STEM*,0.0,0.076102,0.081681,0.0,7.0,34.0,0.205882,STEM*,6.996078,106.547539,0.065662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44458,3.0,29.0,1578115.0,Selma Björnsdóttir,0.122533,Q175119,Culture.Performing_arts,0.5,0.068307,0.073372,77.0,2.0,117.0,0.017094,Performing_arts,0.735199,3150.654032,0.000233
35758,1.0,29.0,562259.0,List of eponymously named diseases,0.228166,Q11758019,History_and_Society.Society,0.0,0.067830,0.072338,77.0,3.0,117.0,0.025641,Society,0.684499,3150.654032,0.000217
14269,1.0,30.0,1882510.0,Wheel gymnastics,0.268951,Q567014,Culture.Performing_arts,0.0,0.064936,0.068902,77.0,2.0,117.0,0.017094,Performing_arts,0.537903,3150.654032,0.000171
40674,1.0,28.0,896415.0,List of political term limits,0.100889,Q6634307,Culture.Sports,0.0,0.073277,0.078442,77.0,5.0,117.0,0.042735,Sports,0.504443,3150.654032,0.000160


,Main_Topics,Higher_Weighted_Article
0,STEM* (0.21) / Biology (0.09) / Business_and_e...,Pacific saury (20)
1,History (0.18) / Military_and_warfare (0.18) /...,Robert the Bruce (660)
2,Politics_and_government (0.27) / Business_and_...,Cynthia Lummis (268)
3,Sports (0.9) / Society (0.02) / STEM* (0.01),Beauden Barrett (1584)
4,STEM* (0.17) / Society (0.15) / Biology (0.08),Neanderthal (752)
...,...,...
73,Military_and_warfare (0.26) / Society (0.19) /...,Northern Fleet (91)
74,Military_and_warfare (0.12) / Politics_and_gov...,New York and New Jersey campaign (1659)
75,Education (0.38) / STEM* (0.12) / Politics_and...,London Metropolitan University (422)
76,Media* (0.45) / Music (0.43) / Entertainment (...,Never Fade (Alice in Chains song) (347)
